# FUNCTION rldesc


The rldesc function computes the run length descriptors from the run length matrix found at [iarl](iarl.ipynb)


<strong> rl,g = rldesc(f,theta=0,phi=0,mask = []) </strong>
<ul>
    <li> <strong> Output </strong> </li>
        <ul> 
            <li> <strong> g: </strong>list of descriptors computed from the iarlrun length.</li>
            <li> <strong> rl: </strong> run length matrix (gray level x run length)</li>
        </ul> 
    <li> <strong> Input </strong> </li>
        <ul>
            <li> <strong> f: </strong> input ndarray (2D or 3D) </li>
            <li> <strong> theta: </strong> angle (rad) in which the run length matrix must be computed. </li>
            <li> <strong> phi: </strong> other angle (rad) in which the run length matrix must be computed (3D cases). </li>
        </ul>
</ul>

The rldesc function computes the descriptors based on the run length matrix.

# FUNCTION CODE


In [3]:
import numpy as np
import nbimporter
from auxiliary_functions import *
from iarl import *


def rldesc(f,theta=0,phi=0,mask = []):

    if mask != []: # if there is a mask
        aux = f.copy()
        new_v = aux[mask].max() +1# value that the pixels outside the mask must assume
        aux[~mask] = new_v # all pixels out of the mask are set with f[mask].max()+1 (maximum value in the mask plus one)
        run_length = iarl(aux,theta,phi)*1.0

        # extract the lines and columns that corresponds to pixels outside the mask with value new_v
        run_length = run_length[:-1,:]# extract the row that represents the pixels out of the mask
        run_length = run_length[:,:np.argwhere(np.sum(run_length,axis=0)>0).max()+1]
        # in some cases, some different lengths of runs are added in this operation
        # on those cases, besides to extracted the row corresponds to new_v
        # it must be extracted the columns added in this operations (i.e. all null columns in the end of the matrix)

    else:
        run_length = iarl(f,theta,phi)
    C = run_length.sum()
    i,j = np.indices(run_length.shape)+1
    # compute descriptors
    ShrtREmph = ((run_length/(j*j)).sum())/C
    LngREmph = ((run_length*(j*j)).sum())/C
    GLevNonUni = (run_length.sum(axis=1)*run_length.sum(axis=1)).sum()/C
    RLNonUni = (run_length.sum(axis=0)*run_length.sum(axis=0)).sum()/C
    Fraction = C/((run_length*j).sum())
    return run_length,[RLNonUni,GLevNonUni,LngREmph,ShrtREmph,Fraction]

# EXAMPLES


Numeric Example:

In the example below, there are 7 runs: 4 runs of length 1 - with values 5, 3, 0 and 2. There are 3 runs of length 2 - 2 with values 2 and 1 with value 5. The matrix has 6 rows, representing the levels and 2 columns representing the length of the runs:

In [2]:
import numpy as np
import nbimporter
from iarl import *

f = np.array([[1,1,5,3,0],
              [1,1,5,5,2]])
print 'Input image'
print f
print 'Run Length Matrix'
print iarl(f)


Input image
[[1 1 5 3 0]
 [1 1 5 5 2]]
Run Length Matrix


IndexError: index 10 is out of bounds for axis 0 with size 6

In [ ]:
print 'Run Length Matrix'
print iarl(f)
print 'Run Length Matrix Descriptors:'
print rldesc(f)

In [ ]:
f = np.array( [[0,1,1,0,0,0,0,0,0],
               [1,0,0,0,0,0,0,1,0],
               [1,0,0,1,0,0,0,1,0],
               [0,0,0,0,0,1,1,0,0]], dtype=np.uint8)
print 'Run Length Matrix'
rl,desc = rldesc(f,np.pi/2)
print rl
print 'Run Length Matrix Descriptors:\n', desc

In [ ]:
f = np.array([[1,1,0,0,0],
              [1,1,8,0,2]])
print 'Run Length Matrix'
rl,desc = rldesc(f,mask = f>0)
print rl
print 'Run Length Matrix Descriptors:\n', desc

Using masks

In [ ]:
f = np.array([[1,1,0,0,9],
              [1,1,8,0,9],
              [1,0,8,0,9],
              [9,9,9,9,9]])
print 'Run Length Matrix'
rl,desc = rldesc(f,mask = f<9)
print rl
print 'Run Length Matrix Descriptors:\n', desc

In [ ]:
f = np.array([[1,0,0,0,1,0],
              [0,1,8,1,1,1],
              [2,2,2,8,2,8],
              [0,1,8,1,1,1],
              [9,9,9,9,9,9],
              [1,1,1,0,8,1]])
print 'Run Length Matrix'
rl,desc = rldesc(f,mask = f<9)
print rl
print 'Run Length Matrix Descriptors:\n', desc
print 
print

3D Examples

In [ ]:
f = np.array([[[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
              [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
              [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]]], dtype=np.uint8)
print 'input array \n',f
rl,desc = rldesc(f) # orientation straight up
print 'rl: \n', rl
print 'Run Length Descriptors\n',desc

Other directions

In [ ]:
rl,desc = rldesc(f,theta=np.pi/4,phi=np.pi/4) # orientation (45,45)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=np.pi/4,phi=np.pi/2) # orientation (45,90)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=np.pi/4,phi=3*np.pi/4) # orientation (45,135)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=np.pi/2,phi=np.pi/4) # orientation (90,45)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=np.pi/2,phi=np.pi/2) # orientation (90,90)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=np.pi/2,phi=3*np.pi/4) # orientation (90,135)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=3*np.pi/4,phi=np.pi/4) # orientation (135,45)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=3*np.pi/4,phi=np.pi/2) # orientation (135,90)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=3*np.pi/4,phi=3*np.pi/4) # orientation (135,135)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=0,phi=np.pi/4) # orientation (0,45)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=0,phi=np.pi/2) # orientation (0,90)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

In [ ]:
rl,desc = rldesc(f,theta=0,phi=3*np.pi/4) # orientation (0,135)
print 'rl: \n', rl
print 'Run Length Descriptors',desc

Using mask:

In [ ]:
rl,desc = rldesc(f,mask = f!=1) # orientation straight up
print 'rl: \n', rl
print 'Run Length Descriptors',desc

# EQUATIONS

For the equations described hereafter let the $p(i,j)$ be the number of the times there is a run of length $j$ having grey level $i$, let $N_g$ be the number of grey levels and $N_r$ be the number of runs.

<strong> Short Run Emphasis Inverse Moments </strong>
$$ ShrtREmph = (\sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_r} \frac{p(i,j)}{j^2})/C$$

<strong> Long Run Emphasis Inverse Moments</strong>
$$ LngREmph = (\sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_r} p(i,j)j^2)/C $$

<strong> Grey Level Nonuniformity </strong>
$$ GLevNonUni = (\sum\limits_{i=1}^{N_g}( \sum\limits_{j=1}^{N_r} p(i,j) )^2)/C$$

<strong> Run Length Nonuniformity</strong>
$$ RLNonUni =  (\sum\limits_{i=1}^{N_g}( \sum\limits_{j=1}^{N_r} p(i,j) )^2)/C $$


<strong> Fraction of Image in Runs</strong>
$$ Fraction = {\sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_r} p(i,j)} / {\sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_r} jp(i,j)} $$


<strong> Coeficient C </strong>
$$ C = \sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_r} p(i,j)$$


# REFERENCE

[Article about RL](http://www.cra.org/Activities/craw_archive/dmp/awards/2004/Xu/Papers/final_submission_paper_452_131_last.pdf)

# SEE ALSO